In [1]:
import pandas as pd
import numpy as np
import random

from sklearn import model_selection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split

In [2]:
avisos = pd.read_csv('Predecir/avisos_detalles_train.csv')
postulantes = pd.read_csv('Predecir/postulantes_datos_train.csv')
postulaciones = pd.read_csv('Predecir/df_postulaciones.csv')

In [3]:
del postulantes['Unnamed: 0']
del avisos['Unnamed: 0']
del postulaciones['Unnamed: 0']

In [4]:
avisos_si = avisos.copy()
avisos_si['se_postulo'] = 1

In [5]:
postulaciones = postulaciones.sample(frac=0.25)

In [6]:
postulaciones = pd.merge(postulaciones, avisos_si, how='inner', on=['idaviso'])

In [7]:
postulaciones = pd.merge(postulaciones, postulantes, how='inner', on=['idpostulante'])

In [8]:
def random_fill(row):
    return np.random.choice(avisos['idaviso'])

In [9]:
while (len(postulaciones[postulaciones["se_postulo"]==1])*0.5 > len(postulaciones[postulaciones["se_postulo"]==0])):
    relleno = postulantes.sample(frac=0.5).copy()
    relleno['idaviso'] = np.nan
    relleno['idaviso'] = relleno.apply(random_fill, axis=1)
    relleno = pd.merge(relleno, avisos, how='inner', on=['idaviso'])
    postulaciones = pd.merge(postulaciones, relleno, how='outer')
    postulaciones = postulaciones.fillna({'se_postulo' : 0})

In [10]:
postulaciones.shape

(2362576, 46)

In [11]:
postulaciones = postulaciones.dropna()

In [12]:
se_postulo = postulaciones['se_postulo']
del postulaciones['se_postulo']
del postulaciones['idaviso']
del postulaciones['idpostulante']
postulaciones.head()

,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,incorp_inmediata,...,total_vistas,vistas_median,cant_dias,nombre,estado,sexo,edad,vistas_postulante,cant_dias_actividad,postu_postulante
0,1,2,45,90,0,1,1,0,0,0,...,5659.0,1.0,86.0,3.0,1.0,1,25.0,50.0,57.0,50.0
2,2,2,23,38,0,0,0,0,1,0,...,798.0,1.0,32.0,3.0,1.0,1,25.0,50.0,57.0,50.0
3,1,1,45,166,0,0,1,1,1,0,...,551.0,1.0,51.0,3.0,1.0,1,25.0,50.0,57.0,50.0
5,1,1,44,1854,0,0,1,0,0,0,...,928.0,2.0,38.0,3.0,1.0,1,25.0,50.0,57.0,50.0
6,1,1,144,1731,0,0,0,0,0,0,...,1231.0,1.0,67.0,3.0,1.0,1,25.0,50.0,57.0,50.0


In [13]:
X_train, X_test, y_train, y_test = train_test_split(postulaciones, se_postulo, test_size=0.20, random_state=7)

In [14]:
seed = 52
num_trees = 30
max_features = 7
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
print(results.mean())

0.8708020483017102


In [15]:
model.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features=7, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [16]:
to_predict  = pd.read_csv('Predecir/test_final_100k.csv')
to_predict  = pd.merge(to_predict, avisos, on = "idaviso",how="left")
to_predict = pd.merge(to_predict, postulantes, on = "idpostulante",how="left")
to_predict = to_predict.drop_duplicates(["idaviso","idpostulante"])

In [17]:
to_predict.shape

(100000, 46)

In [18]:
del to_predict["idaviso"]
del to_predict["idpostulante"]
to_predict = to_predict.dropna()

In [19]:
to_sumit = model.predict(to_predict.drop("id",axis = "columns"))

In [20]:
to_predict["prediccion"] = to_sumit

In [21]:
res  =  pd.read_csv('Predecir/test_final_100k.csv')

res = pd.merge(res, avisos, on = "idaviso",how="left")
res = pd.merge(res, postulantes, on = "idpostulante",how="left")
res = res.drop_duplicates(["idaviso","idpostulante"])

In [22]:
to_sumit = pd.merge(res,to_predict,on="id",how="left")["prediccion"]
to_sumit = to_sumit.fillna(random.getrandbits(1))
df = pd.DataFrame()
df["sepostulo"] = to_sumit
df["id"] = pd.read_csv('Predecir/test_final_100k.csv')["id"]
final = df.set_index("id")
final.shape

(100000, 1)

In [23]:
final.to_csv("final2.csv")